# Tampa Bay Tech Events List generator

## Imports

In [ ]:
import os
from datetime import datetime, timedelta
from time import mktime, sleep
from urllib.parse import urlparse, urlunparse

import ipywidgets as widgets
import pyperclip
from dotenv import load_dotenv
from IPython.display import Audio, display
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager

## Section 1: Create the blog post

The first step in creating the weekly list of Tampa Bay tech events is to create a new post in _Global Nerdy_ and generate its basic content, which includes:

- **Blog post title.** Always has the same format, featuring the start and end dates of the week in question: *Tampa Bay tech, entrepreneur, and nerd events list (Monday, {month} {date} – Sunday, {month}, {date})*.
- **A blank line for the hero image.** This is usually a picture of some place in the Tampa Bay area, with the title _Tampa Bay Tech, Entrepreneur, and Nerd Events_. I make the hero image in Canva, upload it to the blog, and then add it to the post. This is the last thing I do before publishing the post. I expect this will remain a manual process for some time.
- **Intro text.** This explains that the post is a list of events for Tampa Bay’s tech scene, and the text includes the dates of the start and end of the week.
- **The _This week’s events_ list.** This is the _This week’s events_ heading, followed by a bullet-point list of the dates of the days of the week. Each date in the list is linked to a corresponding anchor so that the reader can click on the _Tuesday_ link and be immediately taken to the post’s _Tuesday_ heading.
- **Date headings.** This is a set of &lt;h3&gt; headings, one for each day of the week. Each heading has a designated space where we’ll paste the table of events for that day.
- **Outro text.** This is the text that appears at the end of each of these lists. It explains how I put the list together and what I consider worthy of including in the list.

In [ ]:
def generate_week_list_and_date_headings(start_date_str=None):

    if start_date_str is None or start_date_str.strip() == "":
        start_date = datetime.now()
    else:
        try:
            start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
        except ValueError:
            print("Invalid date format. Please use YYYY-MM-DD.")
            return

    # Ensure the start date is next Monday; if not, move it to the next Monday
    while start_date.weekday() != 0:  # 0 corresponds to Monday
        start_date += timedelta(days=1)

    # Print dates from Monday to Sunday and generate abbreviated strings
    bullet_list = "<ul>\n"
    headings_list = ""
   
    for i in range(7):
        current_date = start_date + timedelta(days=i)
        full_date_str = current_date.strftime("%A, %B %d") # On Windows, the format string should be "%A, %B %#d"
        abbr_date_str = current_date.strftime("%a-%b-%d").lower()
       
        bullet_list += f"""<li><a href="#{abbr_date_str}">{full_date_str}</a></li>\n"""
        headings_list += f"""<a name="{abbr_date_str}"></a>\n<h3>{full_date_str}</h3>\nddd\n\n"""

    bullet_list += "</ul>"

    return f"{bullet_list}\n\n{headings_list}"

## Section 2: Scrape Meetup.com and generate the tables

### Open a Selenium-controlled browser window

In [41]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.get('https://www.facebook.com/v11.0/dialog/oauth?client_id=2403839689&redirect_uri=https%3A%2F%2Fwww.meetup.com%2Fties2%2F&scope=email%20user_friends&response_type=token&state=returnUri%3Dhttps%253A%252F%252Fwww.meetup.com%252Fhome%26facebook%3Dtrue')

load_dotenv()
USERNAME = os.getenv("USERNAME")
PASSWORD = os.getenv("PASSWORD")

username_field = driver.find_element(By.ID, "email")
username_field.send_keys(USERNAME)
sleep(2)
password_field = driver.find_element(By.ID, "pass")
password_field.send_keys(PASSWORD)
sleep(2)
login_button = driver.find_element(By.ID, "loginbutton")
login_button.click()
sleep(8)
continue_button = driver.find_element(By.CLASS_NAME, "x9f619")
continue_button.click()

### Retrieve the list of group and meetup names to ignore

In [42]:
def remove_events_with_ignore_names(events):
    with open("./ignore_names.txt") as ignore_names_file:
        raw_ignore_names = ignore_names_file.readlines()
    NAMES_TO_IGNORE = [raw_ignore_name.strip().lower() for raw_ignore_name in raw_ignore_names]

    result_list = []
    
    for event in events:
        is_in_list = True
        for name_to_ignore in NAMES_TO_IGNORE:
            if name_to_ignore in event['group_name'].lower() or name_to_ignore in event['event_name'].lower():
                is_in_list = False
                break
        if is_in_list:
            result_list.append(event)
        
    return result_list

### The checklist generator: _Run me after logging in!_

In [43]:
def event_urls_from_category_or_keyword_page(category_page_url):
    """
    Given the URL of a Meetup category page, this function returns a list
    of the URLs of the event pages listed on that category page.
    """
    
    def remove_query_parameters(url):
        parsed_url = urlparse(url)
        clean_url = urlunparse((parsed_url.scheme, parsed_url.netloc, parsed_url.path, parsed_url.params, '', ''))
        return clean_url
    
    CONTAINER_ELEMENT_CSS_SELECTOR = "div[class='max-w-narrow']"
    CONTAINER_ELEMENT_CSS_SELECTOR_ALTERNATE = "div[class='grid grid-cols-1 gap-3 sm:grid-cols-2 sm:gap-6 sm:px-6 sm:pt-0 lg:grid-cols-3 xl:grid-cols-4']"
    EVENT_ELEMENT_CSS_SELECTOR = "div[class='flex w-full flex-col items-center']"
    
    event_urls = []

    driver.get(category_page_url)
    wait = WebDriverWait(driver, 2)
    
    # The container element is a <div> containing one or more event items.
    try:
        container_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.max-w-narrow > div")))
        print(f"Found CONTAINER_ELEMENT — tag: {container_element.tag_name}, class: {container_element.get_attribute('class')}")
    except:
        try:
            container_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, CONTAINER_ELEMENT_CSS_SELECTOR_ALTERNATE)))
            print(f"Found CONTAINER_ELEMENT — tag: {container_element.tag_name}, class: {container_element.get_attribute('class')}")
        except:
            print("Failed to find main element.")
            return []
    
    event_elements = container_element.find_elements(By.CSS_SELECTOR, EVENT_ELEMENT_CSS_SELECTOR)
    elements_count = len(event_elements)
    print(f"Found {elements_count} elements.")

    for event_element in event_elements:
        event_link_element = event_element.find_element(By.TAG_NAME, "a")
        event_url = event_link_element.get_attribute("href")
        event_urls.append(remove_query_parameters(event_url))
        
    return list(set(event_urls))

def event_details_from_event_page(event_url):
    """
    Given the URL of a Meetup event page, this function returns a dictionary
    containing the following data from that page:

    - url
    - event_name
    - group_name
    - location
    - time
    - datetime
    """
    event_dict = {
        'event_url'    : event_url,
        'event_name'   : "",
        'group_name'   : "",
        'group_url'    : "",
        'location'     : "",
        'display_time' : "",
        'datetime'     : "",
    }

    driver.get(event_url)
    sleep(5)

    EVENT_NAME_ELEMENT_CSS_SELECTOR = "h1[class='overflow-hidden overflow-ellipsis text-3xl font-bold leading-snug']"
    GROUP_NAME_LINK_ELEMENT_CSS_SELECTOR = "a[id='event-group-mobile-link']"
    GROUP_NAME_ELEMENT_CSS_SELECTOR = "div[class='w-4/5 text-xl font-semibold false']"
    LOCATION_ELEMENT_CSS_SELECTOR = "a[class='hover:text-viridian hover:no-underline']"
    DISPLAY_TIME_ELEMENT_CSS_SELECTOR = "time[class='block']"

    try:
        event_name = driver.find_element(By.CSS_SELECTOR, EVENT_NAME_ELEMENT_CSS_SELECTOR).text
        event_dict['event_name'] = event_name
    except:
        print(f"Couldn’t find event name element for event at {event_url}.")
        print("Returning empty dict.")
        return {}

    try:
        group_name_link_element = driver.find_element(By.CSS_SELECTOR, GROUP_NAME_LINK_ELEMENT_CSS_SELECTOR)
        group_name_element = driver.find_element(By.CSS_SELECTOR, GROUP_NAME_ELEMENT_CSS_SELECTOR)
    except:
        print(f"Couldn’t find group name link element or group name element for event at {event_url}.")
        print("Returning empty dict.")
        return {}
    else:
        try:
            group_name = group_name_element.text.splitlines()[0]
        except:
            group_name = " "
        event_dict['group_name'] = group_name
        group_url = group_name_link_element.get_attribute('href')
        event_dict['group_url'] = group_url
        
    try:
        location_element = driver.find_element(By.CSS_SELECTOR, LOCATION_ELEMENT_CSS_SELECTOR)
        location = location_element.text
    except:
        location = "Online"
    event_dict['location'] = location
    
    try:
        display_time_element = driver.find_element(By.CSS_SELECTOR, DISPLAY_TIME_ELEMENT_CSS_SELECTOR)
    except:
        print(f"Couldn’t find display time element for event at {event_url}.")
        print("Returning empty dict.")
        return {}
    else:
        try:
            display_time = display_time_element.text.splitlines()[1]
        except:
            display_time = display_time_element.text
    event_dict['display_time'] = display_time
    datetime = display_time_element.get_attribute('datetime')
    event_dict['datetime'] = datetime

    print(f"{event_dict}\n")
    return event_dict

def meetup_events(year, month, day):
    events = []
    
    BASE_URL = 'https://www.meetup.com/find'
    KEYWORDS = {
        'programming':             'programming',
        'data%20science':          'data science',
        'project%20management':    'project management',
        'security':                'security',
        'cryptocurrency':          'cryptocurrency',
        'cyber':                   'cyber',
        'agile':                   'agile',
        'entrepreneur':            'entrepreneur',
        'startup':                 'startup',
        'artificial intelligence': 'artificial intelligence'
    }
    CATEGORIES = {
        '546': 'Technology',
        '405': 'Career & Business',
        '604': 'Community & Environment',
        '535': 'Games',
        '571': 'Hobbies & Passions',
        '436': 'Science & Education',
        '652': 'Social Activities',
        '467': 'Writing',
    }

    url_date = f'{year}-{month:02d}-{day:02d}'
    start_date_parameter = f'customStartDate={url_date}T00%3A00-05%3A00'
    end_date_parameter = f'customEndDate={url_date}T23%3A59-05%3A00'
    parameters = f'source=EVENTS&{start_date_parameter}&{end_date_parameter}&distance=hundredMiles&location=us--fl--Tampa'

    # for keyword in KEYWORDS:
    #     print(f"Reading {KEYWORDS[keyword]} keyword page...")
    #     keyword_page_url = f'{BASE_URL}/?{parameters}&keywords={keyword}'
    #     event_urls = event_urls_from_category_or_keyword_page(keyword_page_url)
    #     for event_url in event_urls:
    #         events.append(event_details_from_event_page(event_url))
        
    for category in CATEGORIES:
        print(f"Reading {CATEGORIES[category]} category page...")
        category_page_url = f'{BASE_URL}/?{parameters}&categoryId={category}'
        event_urls = event_urls_from_category_or_keyword_page(category_page_url)
        for event_url in event_urls:
            events.append(event_details_from_event_page(event_url))
        
    return events

def sorted_events(events):
    """
    Given a list of event objects, this method returns a new list
    containing the event objects sorted in chronological order,
    based on the datetime value in each event’s 'datetime' key.
    """
    return sorted(events, key=lambda event:event['datetime'])

def event_checkbox_description(event):
    return f"{event['group_name']}: {event['event_name']}\n" + \
           f"{event['display_time']}\n"

def build_checklist(events):
    checklist = {}

    for event in events:
        checkbox = widgets.Checkbox(
            value = True,
            description = event_checkbox_description(event),
            layout=widgets.Layout(width="800px")
        )
        checklist[checkbox] = event
        
    return checklist

def display_checklist(checklist):
    for item in checklist:
        
        event = checklist[item]
        url = event['event_url']
        link = widgets.HTML(
            value = f"<a href={url} target=\"_blank\">link</a>"
        )
        
        display(widgets.HBox([item, link]))


def remove_duplicate_events(events):
    result_event_urls = []
    result_events = []
    
    for event in events:
        if event['event_url'] in result_event_urls:
            continue
        else:
            result_event_urls.append(event['event_url'])
            result_events.append(event)
            
    return result_events
        
def generate_checklist(year, month, day):
    print("generate_checklist()")
    global checklist
    
    # date = date_picker.value
    # year = date.year
    # month = date.month
    # day = date.day
    
    initial_events = meetup_events(year, month, day) #+ eventbrite_events(year, month, day)
    print("Generated initial events")
    
    sorted_filtered_events = sorted_events(
        remove_events_with_ignore_names(remove_duplicate_events(initial_events))
    )
    checklist = build_checklist(sorted_filtered_events)
    display_checklist(checklist)

In [45]:
generate_checklist(2025, 6, 22)

generate_checklist()
Reading Technology category page...
Found CONTAINER_ELEMENT — tag: div, class: grid grid-cols-1 gap-3 sm:grid-cols-2 sm:gap-6 sm:px-6 sm:pt-0 lg:grid-cols-3 xl:grid-cols-4
Found 4 elements.
{'event_url': 'https://www.meetup.com/makerspaces-pinellas-meetup-group/events/308131954/', 'event_name': 'Intro to Sewing', 'group_name': 'Makerspaces Pinellas Meetup Group', 'group_url': 'https://www.meetup.com/makerspaces-pinellas-meetup-group/?eventOrigin=event_home_page', 'location': 'MakerSpace St. Petersburg', 'display_time': '2:00 PM to 4:00 PM EDT', 'datetime': '2025-06-22T14:00:00-04:00'}

{'event_url': 'https://www.meetup.com/tampa-hackerspace/events/308244231/', 'event_name': 'Sew Awesome! (Textile Arts & Crafts)', 'group_name': 'Tampa Hackerspace', 'group_url': 'https://www.meetup.com/tampa-hackerspace/?eventOrigin=event_home_page', 'location': 'Tampa Hackerspace West', 'display_time': '5:30 PM to 8:30 PM EDT', 'datetime': '2025-06-22T17:30:00-04:00'}

{'event_url':

### The table generator: _Run after checking the checklist!_

In [47]:
def get_checked_items(checklist):
    checked_items = []
    
    for checkbox in checklist:
        if checkbox.value:
            checked_items.append(checklist[checkbox])
            
    return checked_items

def checked_items_to_html_table(checked_items):
    event_html_table = """<table><tr><th>Event name and location</th><th>Group</th><th width="20%">Time</th></tr>"""
    
    for event in checked_items:
        event_html_table += f"""<tr><td><strong><a href=\"{event['event_url']}\">{event['event_name']}</a></strong><br /><small>{event['location']}</small></p></td><td><a href=\"{event['group_url']}\">{event['group_name']}</a></td><td><small>{event['display_time']}</small></td></tr>"""
    
    event_html_table += """<tr><td colspan="3"><a href="#top">Return to the top of the list</a></td></tr></table>"""
    
    pyperclip.copy(event_html_table)
    return event_html_table

def checked_items_to_unordered_list(checked_items):
    event_unordered_list = "<ul>"
    
    for event in checked_items:
        event_unordered_list += f"<li>{event['display_time']}: <strong><a href=\"{event['event_url']}\">{event['event_name']}</a></strong> ({event['location']}) - <a href=\"{event['group_url']}\">{event['group_name']}</a></li>\n"

    event_unordered_list += "</ul>"
    pyperclip.copy(event_unordered_list)
    return event_unordered_list
    
table = checked_items_to_html_table(get_checked_items(checklist))
table

'<table><tr><th>Event name and location</th><th>Group</th><th width="20%">Time</th></tr><tr><td><strong><a href="https://www.meetup.com/gengeek/events/308230931/">28 Years Later @</a></strong><br /><small>AMC Veterans 24</small></p></td><td><a href="https://www.meetup.com/gengeek/?eventOrigin=event_home_page">Gen Geek</a></td><td><small>10:00 AM to 12:00 PM EDT</small></td></tr><tr><td><strong><a href="https://www.meetup.com/srqgames/events/308232975/">Venice Strategy Board Gamers</a></strong><br /><small>Venice</small></p></td><td><a href="https://www.meetup.com/srqgames/?eventOrigin=event_home_page">Board Games and Card Games in Sarasota & Bradenton</a></td><td><small>12:00 PM to 6:00 PM EDT</small></td></tr><tr><td><strong><a href="https://www.meetup.com/srq-newbie-mahjong-gathering/events/307773813/">JUNE DOT, CRAK, BAM GATHERING</a></strong><br /><small>The GoodEarth</small></p></td><td><a href="https://www.meetup.com/srq-newbie-mahjong-gathering/?eventOrigin=event_home_page">SRQ 